# Pesticide Informativ on Coffe Culture in Brazil

In [60]:
# imports

import os
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
import csv
import json

In [61]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging

load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')


if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    

OpenAI API Key exists and begins sk-proj-


In [62]:
# Initialize

openai = OpenAI()
MODEL = 'gpt-4o-mini'

In [63]:
system_message = "You should work, mainly, with the tool passed to you"
system_message += "It is a tool based on the pesticides used in coffe harvest in Sao Paulo, Brazil"
system_message += "When you found the tool data, you should reply about all the pesticides and their damage"


In [64]:
# Load pesticide data from a CSV file
def load_pesticide_data(file_name):
    pesticide_data = {}
    try:
        with open(file_name, mode='r', encoding='utf-8') as file:
            reader = csv.reader(file)
            # Assuming the CSV has columns: Year, Food (English), State, Pesticide, Environmental Impact
            next(reader)  # Skip the header row
            for row in reader:
                if len(row) >= 5:
                    pesticide = row[3].strip().lower()  # Use Pesticide as the key
                    environmental_impact = row[4].strip()
                    pesticide_data[pesticide] = environmental_impact
    except FileNotFoundError:
        print(f"Error: The file '{file_name}' was not found.")
    except Exception as e:
        print(f"An error occurred: {e}")
    return pesticide_data

In [65]:

# Initialize pesticide data by reading the CSV
pesticide_data = load_pesticide_data("coffePesticide.csv")

In [66]:



# Function to get environmental impact of a pesticide
def get_environmental_impact(pesticide_name):
    print(f"Tool get_environmental_impact called for {pesticide_name}")
    pesticide = pesticide_name.lower()
    return pesticide_data.get(pesticide, "Unknown")

# Define the function structure for tools
impact_function = {
    "name": "get_environmental_impact",
    "description": "Get the environmental impact of a pesticide. Call this whenever you need to know if a pesticide is hazardous, very hazardous, or has another impact level.",
    "parameters": {
        "type": "object",
        "properties": {
            "pesticide_name": {
                "type": "string",
                "description": "The name of the pesticide\ to look up.",
            },
        },
        "required": ["pesticide_name"],
        "additionalProperties": False
    }
}

# List of tools
tools = [{"type": "function", "function": impact_function}]



In [67]:
# Example call
print(get_environmental_impact("GLIFOSATO"))

Tool get_environmental_impact called for GLIFOSATO
Hazardous


In [68]:
# Include the function in the list of tools
tools = [{"type": "function", "function": impact_function}]  # External data provided to OpenAI

# Example: Check the tools list
print(tools)

[{'type': 'function', 'function': {'name': 'get_environmental_impact', 'description': 'Get the environmental impact of a pesticide. Call this whenever you need to know if a pesticide is hazardous, very hazardous, or has another impact level.', 'parameters': {'type': 'object', 'properties': {'pesticide_name': {'type': 'string', 'description': 'The name of the pesticide\\ to look up.'}}, 'required': ['pesticide_name'], 'additionalProperties': False}}}]


In [75]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}]
    messages.extend(history)
    messages.append({"role": "user", "content": message})
    
    try:
        # First check if we need to make a tool call
        response = openai.chat.completions.create(
            model=MODEL,
            messages=messages,
            tools=tools
        )
        
        if response.choices[0].finish_reason == "tool_calls":
            print("🔧 Tool call detected! Processing tool request...")
            tool_message = response.choices[0].message
            tool_response, pesticide_name = handle_tool_call(tool_message)
            
            # Append tool call messages
            messages.extend([
                {"role": "assistant", "content": None, "tool_calls": tool_message.tool_calls},
                tool_response
            ])
            
            # Stream the final response with tool results
            stream = openai.chat.completions.create(
                model=MODEL,
                messages=messages,
                stream=True
            )
            
            partial_message = ""
            for chunk in stream:
                if chunk.choices[0].delta.content is not None:
                    partial_message += chunk.choices[0].delta.content
                    yield partial_message
                    
        else:
            print("✨ No tool call needed - streaming direct response")
            # Stream the direct response
            stream = openai.chat.completions.create(
                model=MODEL,
                messages=messages,
                stream=True
            )
            
            partial_message = ""
            for chunk in stream:
                if chunk.choices[0].delta.content is not None:
                    partial_message += chunk.choices[0].delta.content
                    yield partial_message
                    
    except Exception as e:
        print(f"Error in chat function: {str(e)}")
        yield f"An error occurred: {str(e)}"

interface = gr.ChatInterface(
    fn=chat,
    chatbot=gr.Chatbot(
        height=300, 
        type="messages",
        bubble_full_width=False,
        show_label=False,
        render_markdown=True
    ),
    textbox=gr.Textbox(
        placeholder="Ask about pesticides...", 
        container=False,
        scale=7
    ),
    title="Pesticide Environmental Impact Assistant",
    description="Ask about the environmental impact of pesticides used in São Paulo coffee harvesting.",
    type="messages",
    examples=[
        "What is the environmental impact of GLIFOSATO?",
        "Tell me about the most hazardous pesticides used in coffee farming."
    ]
)

# Launch the interface
interface.launch()

* Running on local URL:  http://127.0.0.1:7886

To create a public link, set `share=True` in `launch()`.


🔧 Tool call detected! Processing tool request...
Tool get_environmental_impact called for Paraquat
🔧 Tool call detected! Processing tool request...
Tool get_environmental_impact called for fosfina
✨ No tool call needed - streaming direct response
✨ No tool call needed - streaming direct response
